In [15]:
%pip install --upgrade geonamescache requests pandas


Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import requests
import geonamescache

# --- Reload cleaned cities ---
df = pd.read_csv('../data/world_cities_updated.csv')

# --- Re-derive log_population for modeling ---
df['log_population'] = np.log1p(df['population'])

# Quick sanity check
df[['population','log_population']].head()


,population,log_population
0,15853,9.671177
1,20430,9.924809
2,62747,11.046882
3,351943,12.771227
4,63482,11.058527


In [10]:
# Build ISO2 → ISO3 lookup via geonamescache
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
df_cnt = pd.DataFrame(countries).T
code_map = df_cnt[['iso','iso3']].set_index('iso')

# Attach ISO3 codes and drop unmapped
df['iso3'] = df['countrycode'].map(code_map['iso3'])
df = df.dropna(subset=['iso3']).reset_index(drop=True)

# Show a sample
df[['name','countrycode','iso3']].head()


,name,countrycode,iso3
0,les Escaldes,AD,AND
1,Andorra la Vella,AD,AND
2,Umm Al Quwain City,AE,ARE
3,Ras Al Khaimah City,AE,ARE
4,Zayed City,AE,ARE


In [11]:
# 1) Grab the World Bank’s master country list
resp_all = requests.get(
    "http://api.worldbank.org/v2/country?format=json&per_page=500"
)
valid_iso3 = {c['id'] for c in resp_all.json()[1]}

# 2) Identify any invalid codes
iso3_list    = df['iso3'].unique().tolist()
invalid_codes = [c for c in iso3_list if c not in valid_iso3]
print("Dropping invalid ISO3 codes:", invalid_codes)

# 3) Keep only the valid ones for GDP fetch
df = df[df['iso3'].isin(valid_iso3)].reset_index(drop=True)


Dropping invalid ISO3 codes: ['AIA', 'ALA', 'BLM', 'BES', 'CCK', 'COK', 'CXR', 'ESH', 'FLK', 'GUF', 'GGY', 'GLP', 'SGS', 'JEY', 'MTQ', 'MSR', 'NFK', 'NIU', 'SPM', 'PCN', 'REU', 'SHN', 'SJM', 'ATF', 'TWN', 'VAT', 'WLF', 'MYT']


In [12]:
# Build request URL
valid_list = df['iso3'].unique().tolist()
url = (
    "http://api.worldbank.org/v2/country/" +
    ";".join(valid_list) +
    "/indicator/NY.GDP.PCAP.CD?date=2022&format=json&per_page=500"
)

# Fetch and parse JSON
resp = requests.get(url)
data = resp.json()[1]  # [0] is metadata

# Build a GDP DataFrame
gdp_df = (
    pd.DataFrame(data)
      .loc[:, ['countryiso3code','value']]
      .rename(columns={'countryiso3code':'iso3','value':'gdp_per_capita'})
)

# Merge back and fill missing
df = df.merge(gdp_df, on='iso3', how='left')
df['gdp_per_capita'] = df['gdp_per_capita'].fillna(df['gdp_per_capita'].median())

# Preview
df[['name','iso3','gdp_per_capita']].head()


,name,iso3,gdp_per_capita
0,les Escaldes,AND,42414.059009
1,Andorra la Vella,AND,42414.059009
2,Umm Al Quwain City,ARE,49899.065298
3,Ras Al Khaimah City,ARE,49899.065298
4,Zayed City,ARE,49899.065298


In [13]:
# Normalize and multiply
lp_norm  = df['log_population']    / df['log_population'].max()
gdp_norm = df['gdp_per_capita']    / df['gdp_per_capita'].max()

df['econ_viability'] = lp_norm * gdp_norm

# Preview the final index
df[['name','countrycode','iso3','econ_viability']].head()


,name,countrycode,iso3,econ_viability
0,les Escaldes,AD,AND,0.107241
1,Andorra la Vella,AD,AND,0.110053
2,Umm Al Quwain City,AE,ARE,0.144113
3,Ras Al Khaimah City,AE,ARE,0.166608
4,Zayed City,AE,ARE,0.144265


In [14]:
df.to_csv('../data/cities_with_econ.csv', index=False)
